<a href="https://colab.research.google.com/github/yang-su2000/Authorship-Identification-with-NLP/blob/main/AML_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/MyDrive/Colab Notebooks/Authorship Identification/")

Mounted at /content/drive


### Load CN 1 Dataset (stage 1)

In [ ]:
import pandas as pd

rows_list = []
with open('Citation Network/CN1.txt', 'r') as f:
  n = int(f.readline())
  for _ in range(n):
    d = {'refs': [], 
         'title': 'untitled', 
         'authors': [], 
         'year': -1, 
         'pub': 'no publication', 
         'index': -1, 
         'abstract': 'no abstract'}
    while True:
      line = f.readline()[:-1]
      if line == '':
        break
      if line[1] == '*':
        d['title'] = line[2:]
      elif line[1] == '@':
        d['authors'] = line[2:].split(',')
      elif line[1] == 't':
        d['year'] = int(line[2:])
      elif line[1] == 'c':
        if len(line) > 2:
          d['pub'] = line[2:]
      elif line[1] == '%':
        d['refs'].append(line[2:])
      elif line[1] == '!':
        d['abstract'] = line[2:]
      else:
        d['index'] = int(line[6:])
    rows_list.append(d)

df = pd.DataFrame(rows_list)
print(df.shape)
df.head()

In [ ]:
df.to_csv('Citation Network/CN1.csv', index=False)

### Load CN 1 Dataset (stage 2)


In [ ]:
import pandas as pd

df = pd.read_csv('Citation Network/CN1.csv', dtype={'ref': str, 'title': str, 'authors': str, 'year': int, 'pub': str, 'index': int, 'abstract': str})
df.head()

,refs,title,authors,year,pub,index,abstract
0,[],Automated Deduction in Geometry: 5th Internati...,"['Hoon Hong', 'Dongming Wang']",2006,no publication,0,no abstract
1,[],A+ Certification Core Hardware (Text & Lab Man...,['Charles J. Brooks'],2003,no publication,1,no abstract
2,[],Performance engineering in industry: current p...,"['Ahmed E. Hassan', 'Parminder Flora']",2007,Proceedings of the 6th international workshop ...,2,This panel session discusses performance engin...
3,[],"Dude, You Can Do It! How to Build a Sweeet PC","['Darrel Creacy', 'Carlito Vicencio']",2005,no publication,3,Whether you're frustrated with current PC offe...
4,[],What Every Programmer Needs to Know about Secu...,"['Neil Daswani', 'Anita Kesavan']",2006,no publication,4,no abstract


In [ ]:
author_set = set()
for ls in df['authors'].values:
  authors = ls[1:-1].split(', ')
  author_set.update(authors)
print(len(author_set))

595775


In [ ]:
df['author_info'] = 'My name is ' + df['authors']

In [ ]:
author_feats = pd.read_csv('Citation Network/author_and_features (1).csv')
author_feats.head()

,Unnamed: 0,authors,affiliation
0,0,Tzu-Chinag Chiang,$PersonNotFound \n
1,1,YUE Xinli,$PersonNotFound \n
2,2,Frank Berger,$PersonNotFound \n
3,3,Jifang Li,$PersonNotFound \n
4,4,Alexandre M. Roma,$PersonNotFound \n


In [ ]:
affiliation_map = {}
for row in author_feats.values:
  author, affiliation = row[1], row[2]
  affiliation_map[author[:-1]] = affiliation[:-1]
print(affiliation_map)
df['affiliation'] = 'No record; '
for i in range(df.shape[0]):
  if i % 1000 == 0:
      print(i, end=' ')
  authors = [author[1:-1] for author in df.iloc[i, 2][1:-1].split(', ')]
  first_author = True
  for author in authors:
    if author in affiliation_map:
      if first_author:
        df.iloc[i, -1] = ''
        first_author = False
      df.iloc[i, -1] += affiliation_map[author] + '; '
df = df[df['affiliation'] != 'No record; ']
df.head()

{'Tzu-Chinag Chiang': ' $PersonNotFound ', 'YUE Xinli': ' $PersonNotFound ', 'Frank Berger': ' $PersonNotFound ', 'Jifang Li': ' $PersonNotFound ', 'Alexandre M. Roma': ' $PersonNotFound ', 'Quanzhong Li': 'Sun Yat-Sen University, Guangzhou, China', 'Marco Bekooij': 'University of York, Department of Computer Science, UK', 'Dileep Kumar Panjwani': ' $PersonNotFound ', 'R. Vile': ' $PersonNotFound ', 'Sven J. Fischer': ' $PersonNotFound ', 'Suiliong Liang': ' $PersonNotFound ', 'G. Raskin': ' $PersonNotFound ', 'Howard L. Bailit': ' $PersonNotFound ', 'Kosin Chamnongthai': ' $PersonNotFound ', 'Sugata Mitra': ' $PersonNotFound ', 'D. Karalekas': ' $PersonNotFound ', 'R. D. Faulk': ' $PersonNotFound ', 'H. Koshinsky': ' $PersonNotFound ', 'Chengmo Yang': 'Duke University, Department of Electrical and Computer Engineering, Durham, NC, USA', 'Andrea Rotnizky': ' $PersonNotFound ', 'Yiping Yao': 'Nanyang Technological University, Division of Computer Science, School of Computer Engineering,

,refs,title,authors,year,pub,index,abstract,author_info,affiliation
105,[],Embedded Software and Systems: Second Internat...,"['Laurence T. Yang', 'Xingshe Zhou', 'Wei Zhao...",2006,no publication,105,no abstract,"My name is ['Laurence T. Yang', 'Xingshe Zhou'...","Luleå University of Technology, Skellefteå, Sw..."
108,[],C++ Cookbook (Cookbooks (O'Reilly)),"['D. Ryan Stephens', 'Christopher Diggins', 'J...",2005,no publication,108,Despite its highly adaptable and flexible natu...,"My name is ['D. Ryan Stephens', 'Christopher D...",$PersonNotFound ;
203,[],Special Edition Using Microsoft Office Word 2003,"['Bill Camarda', 'Neil Randall']",2003,no publication,203,"More than any other, this book offers systemat...","My name is ['Bill Camarda', 'Neil Randall']",$PersonNotFound ;
225,"['437914', '300194', '100279', '446382', '4481...",ALP: Efficient support for all levels of paral...,"['Ruchira Sasanka', 'Man-Lap Li', 'Sarita V. A...",2007,ACM Transactions on Architecture and Code Opti...,225,The real-time execution of contemporary comple...,"My name is ['Ruchira Sasanka', 'Man-Lap Li', '...",$PersonNotFound ;
312,[],ActionScript 2.0 Language Reference for Macrom...,"['Francis Cheng', 'Jen deHaan', 'Robert L. Dix...",2005,no publication,312,The ActionScript 2.0 Language Reference for Ma...,"My name is ['Francis Cheng', 'Jen deHaan', 'Ro...",$PersonNotFound ;


In [ ]:
df.to_csv('Citation Network/CN1_affiliated.csv', index=False)

In [ ]:
df.shape, df[df['affiliation'] == 'No record; '].shape

((22282, 9), (0, 9))

In [ ]:
df.sort_values(by='affiliation', ascending=False).head()

,refs,title,authors,year,pub,index,abstract,author_info,affiliation
102127,"['520984', '586775', '307205', '319954', '2332...",Multiregion Level-Set Partitioning of Syntheti...,"['Ismail Ben Ayed', 'Amar Mitiche', 'Ziad Belh...",2005,IEEE Transactions on Pattern Analysis and Mach...,102127,The purpose of this study is to investigate Sy...,"My name is ['Ismail Ben Ayed', 'Amar Mitiche',...","École de technologie supérieure, Montréal, Can..."
24565,"['307205', '102127']",Polarimetric Image Segmentation via Maximum-Li...,"['Ismail Ben Ayed', 'Amar Mitiche', 'Ziad Belh...",2006,IEEE Transactions on Pattern Analysis and Mach...,24565,This study investigates a level set method for...,"My name is ['Ismail Ben Ayed', 'Amar Mitiche',...","École de technologie supérieure, Montréal, Can..."
36228,"['317058', '265639', '314663']",Curry-style type isomorphisms and game semantics,['Joachim De lataillade'],2008,Mathematical Structures in Computer Science,36228,"Curry-style system F, that is, system F with n...",My name is ['Joachim De lataillade'],"École Polytechnique, Palaiseau, France;"
217984,"['518244', '542576', '523200', '455989', '5286...",Teaching mathematics using technology strategies,['Robert Lipton'],1994,ACM SIGCUE Outlook,217984,This article explores alternate ways that tech...,My name is ['Robert Lipton'],"École Polytechnique de Montréal, Québec, Canada;"
476521,[],Laminar elastic composites with crystallograph...,"['Richard James', 'Robert Lipton', 'Adam Lutob...",1990,SIAM Journal on Applied Mathematics,476521,no abstract,"My name is ['Richard James', 'Robert Lipton', ...","École Polytechnique de Montréal, Québec, Canada;"


### Build Pos-Neg Pair For Contrastive Learning

In [ ]:
import numpy as np
import pandas as pd

random_seed = 42
np.random.seed = random_seed

def build_pair(pos_df, PosToNeg_ratio):
  posN = pos_df.shape[0]
  negN = int(posN * PosToNeg_ratio)
  neg_df = df.sample(negN, replace=True, random_state=random_seed)
  affiliation_list = list(set(affiliation_map.values()))
  for i in range(negN):
    if i % 1000 == 0:
      print(i, end=' ')
    pos_affiliation = neg_df.iloc[i, -1]
    while True:
      neg_affiliation = np.random.choice(affiliation_list)
      if neg_affiliation in pos_affiliation:
        continue
      neg_df.iloc[i, -1] = neg_affiliation + '; '
      break
  pos_df['label'] = 1
  neg_df['label'] = 0
  return pd.concat([pos_df, neg_df])

df = pd.read_csv('Citation Network/CN1_affiliated.csv')
df = build_pair(df, PosToNeg_ratio=3.0)
print(df.shape)
df.head()

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 (89128, 10)


,refs,title,authors,year,pub,index,abstract,author_info,affiliation,label
0,[],Embedded Software and Systems: Second Internat...,"['Laurence T. Yang', 'Xingshe Zhou', 'Wei Zhao...",2006,no publication,105,no abstract,"My name is ['Laurence T. Yang', 'Xingshe Zhou'...","Luleå University of Technology, Skellefteå, Sw...",1
1,[],C++ Cookbook (Cookbooks (O'Reilly)),"['D. Ryan Stephens', 'Christopher Diggins', 'J...",2005,no publication,108,Despite its highly adaptable and flexible natu...,"My name is ['D. Ryan Stephens', 'Christopher D...",$PersonNotFound ;,1
2,[],Special Edition Using Microsoft Office Word 2003,"['Bill Camarda', 'Neil Randall']",2003,no publication,203,"More than any other, this book offers systemat...","My name is ['Bill Camarda', 'Neil Randall']",$PersonNotFound ;,1
3,"['437914', '300194', '100279', '446382', '4481...",ALP: Efficient support for all levels of paral...,"['Ruchira Sasanka', 'Man-Lap Li', 'Sarita V. A...",2007,ACM Transactions on Architecture and Code Opti...,225,The real-time execution of contemporary comple...,"My name is ['Ruchira Sasanka', 'Man-Lap Li', '...",$PersonNotFound ;,1
4,[],ActionScript 2.0 Language Reference for Macrom...,"['Francis Cheng', 'Jen deHaan', 'Robert L. Dix...",2005,no publication,312,The ActionScript 2.0 Language Reference for Ma...,"My name is ['Francis Cheng', 'Jen deHaan', 'Ro...",$PersonNotFound ;,1


In [ ]:
df.shape, df[df['affiliation'] == ' $PersonNotFound ; '].shape

((89128, 10), (12979, 10))

In [ ]:
df.to_csv('Citation Network/CN1_affiliated_labeled_90k.csv', index=False)

### Load CN1 Dataset (final stage)

In [ ]:
import pandas as pd

df = pd.read_csv('Citation Network/CN1_affiliated_labeled_90k.csv', 
                 dtype={'ref': str, 'title': str, 'authors': str, 'year': int, 'pub': str, 'index': int, 'abstract': str, 'affiliation': str, 'label': int})
df.head()

,refs,title,authors,year,pub,index,abstract,author_info,affiliation,label
0,[],Embedded Software and Systems: Second Internat...,"['Laurence T. Yang', 'Xingshe Zhou', 'Wei Zhao...",2006,no publication,105,no abstract,"My name is ['Laurence T. Yang', 'Xingshe Zhou'...","Luleå University of Technology, Skellefteå, Sw...",1
1,[],C++ Cookbook (Cookbooks (O'Reilly)),"['D. Ryan Stephens', 'Christopher Diggins', 'J...",2005,no publication,108,Despite its highly adaptable and flexible natu...,"My name is ['D. Ryan Stephens', 'Christopher D...",$PersonNotFound ;,1
2,[],Special Edition Using Microsoft Office Word 2003,"['Bill Camarda', 'Neil Randall']",2003,no publication,203,"More than any other, this book offers systemat...","My name is ['Bill Camarda', 'Neil Randall']",$PersonNotFound ;,1
3,"['437914', '300194', '100279', '446382', '4481...",ALP: Efficient support for all levels of paral...,"['Ruchira Sasanka', 'Man-Lap Li', 'Sarita V. A...",2007,ACM Transactions on Architecture and Code Opti...,225,The real-time execution of contemporary comple...,"My name is ['Ruchira Sasanka', 'Man-Lap Li', '...",$PersonNotFound ;,1
4,[],ActionScript 2.0 Language Reference for Macrom...,"['Francis Cheng', 'Jen deHaan', 'Robert L. Dix...",2005,no publication,312,The ActionScript 2.0 Language Reference for Ma...,"My name is ['Francis Cheng', 'Jen deHaan', 'Ro...",$PersonNotFound ;,1


### Preprocessing

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 14.5 MB/s 
     |████████████████████████████████| 7.6 MB 64.9 MB/s 
     |████████████████████████████████| 163 kB 52.6 MB/s 


In [ ]:
import torch
from sklearn import preprocessing
from transformers import DistilBertTokenizer, BertTokenizer, DebertaTokenizer

random_seed = 42
test_frac = 0.2
author_max_length = 16
title_max_length = 32
pub_max_length = 16
abstract_max_length = 128
valid_frac = 0.1 / (1 - test_frac)
test_df = df.sample(frac=test_frac, random_state=random_seed)
train_df = df.drop(test_df.index)

le = preprocessing.LabelEncoder()
le.fit(train_df['label'])
train_df['label'] = le.transform(train_df['label']).astype(int)
test_df['label'] = le.transform(test_df['label']).astype(int)

valid_df = train_df.sample(frac=valid_frac, random_state=random_seed)
train_df = train_df.drop(valid_df.index)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
train_df.shape, valid_df.shape, test_df.shape

((62389, 10), (8913, 10), (17826, 10))

In [ ]:
train_df[train_df['label'] == 1].shape, valid_df[valid_df['label'] == 1].shape, test_df[test_df['label'] == 1].shape

((15585, 10), (2252, 10), (4445, 10))

In [ ]:
test_df.head()

,refs,title,authors,year,pub,index,abstract,author_info,affiliation,label
77002,[],Supporting units of work in a virtual organiza...,['Bill Chu'],1995,ACM SIGOIS Bulletin,585074,no abstract,My name is ['Bill Chu'],"University of Alabama, Huntsville, USA;",0
57845,[],Derived relations with exceptions: decision su...,"['R. Dattero', 'R. G. Ramirez', 'J. Choobineh']",1990,Journal of Management Information Systems,472404,no abstract,"My name is ['R. Dattero', 'R. G. Ramirez', 'J....","Yonsei University, Department of Computer Scie...",0
71770,"['108547', '309165']",Grid'5000: A Large Scale and Highly Reconfigur...,"['F. Cappello', 'E. Caron', 'M. Dayde', 'F. De...",2005,Proceedings of the 6th IEEE/ACM International ...,426847,no abstract,"My name is ['F. Cappello', 'E. Caron', 'M. Day...","New York University, Stern School of Business,...",0
41647,[],Proceedings of the ACM SIGPLAN 1993 conference...,['Robert Cartwright'],1993,Conference on Programming Language Design and ...,217856,no abstract,My name is ['Robert Cartwright'],"California University of Pennsylvania, Departm...",0
75510,[],"Ad-Hoc, Mobile, and Wireless Networks: Second ...","['Samuel Pierre', 'Michel Barbeau', 'Evangelos...",2003,no publication,5382,no abstract,"My name is ['Samuel Pierre', 'Michel Barbeau',...","Xidian University, Department of Information E...",0


In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.label = df['label'].values
        self.author_info = [tokenizer(text, padding='max_length', max_length = author_max_length, truncation=True, return_tensors="pt") for text in 
                            df['affiliation'] + df['author_info']]
        self.title = [tokenizer(text, padding='max_length', max_length = title_max_length, truncation=True, return_tensors="pt") for text in df['title']]
        self.pub = [tokenizer(text, padding='max_length', max_length = pub_max_length, truncation=True, return_tensors="pt") for text in df['pub']]
        self.abstract = [tokenizer(text, padding='max_length', max_length = abstract_max_length, truncation=True, return_tensors="pt") for text in df['abstract']]

    def classes(self):
        return self.label

    def __len__(self):
        return len(self.label)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.label[idx])

    def get_batch_author_infos(self, idx):
        # Fetch a batch of author infos
        return self.author_info[idx]

    def get_batch_titles(self, idx):
        # Fetch a batch of titles
        return self.title[idx]

    def __getitem__(self, idx):

        batch_titles = self.get_batch_titles(idx)
        batch_author_infos = self.get_batch_author_infos(idx)
        batch_labels = self.get_batch_labels(idx)
        return batch_titles, batch_author_infos, batch_labels

### Baseline Model Class

In [ ]:
from torch import nn
from transformers import DistilBertModel, BertModel, DebertaModel, get_linear_schedule_with_warmup

class BaselineModel(nn.Module):

    def __init__(self, dropout=0.5, encoder_hidden=32, decoder_hidden=len(le.classes_)):

        super().__init__()

        self.encoder_hidden = encoder_hidden
        self.decoder_hidden = decoder_hidden
        self.text_encoder = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.author_encoder = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear_encoder = nn.Linear(768, encoder_hidden)
        self.text_decoder = nn.Linear(encoder_hidden * title_max_length, decoder_hidden * encoder_hidden)
        self.author_decoder = nn.Linear(encoder_hidden * author_max_length, decoder_hidden * encoder_hidden)
        self.cos_similarity = nn.CosineSimilarity(dim=-1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self, text_ids, author_ids):

        text_output = self.text_encoder(input_ids=text_ids).last_hidden_state
        author_output = self.author_encoder(input_ids=author_ids).last_hidden_state
        text_encoded = self.relu(self.linear_encoder(self.dropout(text_output)))
        author_encoded = self.relu(self.linear_encoder(self.dropout(author_output)))
        text_decoded = self.text_decoder(text_encoded.flatten(1)).view(text_encoded.shape[0], self.decoder_hidden, self.encoder_hidden)
        author_decoded = self.author_decoder(author_encoded.flatten(1)).view(author_encoded.shape[0], self.decoder_hidden, self.encoder_hidden)
        similarity = self.cos_similarity(text_decoded, author_decoded)
        output = self.sigmoid(similarity)
        return output

### Training

In [ ]:
# !git clone https://gist.github.com/NTT123/4596e5533e573c8ceab2f319ab5d36a2 jslog
import random
import math
import time
from jslog.jslogger import JSLogger

logger = JSLogger('train/valid loss', ['train', 'valid'])
logger_ = JSLogger('train/valid accuracy', ['train', 'valid'])

In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=8, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=8)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=len(train_dataloader) * epochs,
    )

    if use_cuda:

            print('cuda version', torch.__version__)
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            model.train()

            for text_input, author_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                # mask = train_input['attention_mask'].to(device)
                text_id = text_input['input_ids'].squeeze(1).to(device)
                author_id = author_input['input_ids'].squeeze(1).to(device)

                output = model(text_id, author_id)
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
                lr_scheduler.step()
            
            total_acc_val = 0
            total_loss_val = 0
            model.eval()

            with torch.no_grad():

                for text_input, author_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    text_id = text_input['input_ids'].squeeze(1).to(device)
                    author_id = author_input['input_ids'].squeeze(1).to(device)

                    output = model(text_id, author_id)
                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .4f} \
                | Train Accuracy: {total_acc_train / len(train_data): .4f} \
                | Val Loss: {total_loss_val / len(val_data): .4f} \
                | Val Accuracy: {total_acc_val / len(val_data): .4f}')
            
            logger.log(epoch_num + 1, {'train': total_loss_train / len(train_data), 'valid': total_loss_val / len(val_data)})
            logger_.log(epoch_num + 1, {'train': total_acc_train / len(train_data), 'valid': total_acc_val / len(val_data)})

logger.show()
logger_.show()             
EPOCHS = 2
model = BaselineModel()
print('There are', sum(p.numel() for p in model.parameters()), 'parameters')
LR = 2e-5
              
train(model, train_df, valid_df, LR, EPOCHS)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- T

There are 132848800 parameters
cuda version 1.12.1+cu113


100%|██████████| 7799/7799 [12:28<00:00, 10.41it/s]


Epochs: 1 | Train Loss:  0.0671                 | Train Accuracy:  0.8979                 | Val Loss:  0.0669                 | Val Accuracy:  0.8991


...

Epochs: 3 | Train Loss:  0.0668                 | Train Accuracy:  0.9005                 | Val Loss:  0.0669                 | Val Accuracy:  0.8991


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 52%|█████▏    | 4050/7799 [06:23<05:54, 10.56it/s]


KeyboardInterrupt: ignored

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=32)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_loss_test = 0
    total_acc_test = 0
    model.eval()
    
    with torch.no_grad():

        for text_input, author_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              text_id = text_input['input_ids'].squeeze(1).to(device)
              author_id = author_input['input_ids'].squeeze(1).to(device)

              output = model(text_id, author_id)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .4f}')
    
evaluate(model, test_df)

Test Accuracy:  0.9026
